# Setup using OpenShift Local

1. Install OpenShift Local
2. Setup OpenShift Local

In [ ]:
%%sh

crc setup
crc start

Wait until OpenShift Local is ready and prepare the environment

In [ ]:
%%sh

eval $(crc oc-env)
oc login -u kubeadmin -p $(crc console -o json | jq .clusterConfig.adminCredentials.password) https://api.crc.testing:6443
podman login -u kubeadmin -p $(oc whoami -t) default-route-openshift-image-registry.apps-crc.testing --tls-verify=false

For MacOS users:

In case you can't push to local OpenShift registry, run the following:

In [ ]:
%%sh

if [  $(uname -m) == "arm64" ]; then
    podman machine ssh -- 'echo "192.168.127.254 default-route-openshift-image-registry.apps-crc.testing" | sudo tee -a  /etc/hosts'
fi

| OpenShift Local Console | https://console-openshift-console.apps-crc.testing/ |
| -- | -- |

## NPM Registry

In [ ]:
%%sh

oc new-project verdaccio

podman pull verdaccio/verdaccio
podman tag docker.io/verdaccio/verdaccio:latest default-route-openshift-image-registry.apps-crc.testing/verdaccio/verdaccio:latest
podman push --tls-verify=false default-route-openshift-image-registry.apps-crc.testing/verdaccio/verdaccio:latest
oc set image-lookup verdaccio

helm repo add verdaccio https://charts.verdaccio.org
helm upgrade -i npm verdaccio/verdaccio -f verdaccio.yaml

Wait for the registry to deploy. Next setup local NPM config:

In [ ]:
%%sh

npm set registry http://verdaccio-verdaccio.apps-crc.testing/
npm adduser --registry http://verdaccio-verdaccio.apps-crc.testing/

| NPM registry | http://verdaccio-verdaccio.apps-crc.testing/ |
| -- | -- |

## Backstage

Let's deploy Backstage from Janus IDP

In [ ]:
%%sh

oc new-project backstage

if [  $(uname -m) == "arm64" ]; then
    podman pull quay.io/tcoufal/backstage-showcase:arm
    podman tag quay.io/tcoufal/backstage-showcase:arm default-route-openshift-image-registry.apps-crc.testing/backstage/backstage:latest
else
    podman pull quay.io/janus-idp/backstage-showcase
    podman tag quay.io/janus-idp/backstage-showcase:latest default-route-openshift-image-registry.apps-crc.testing/backstage/backstage:latest
fi

podman push --tls-verify=false default-route-openshift-image-registry.apps-crc.testing/backstage/backstage:latest
oc set image-lookup backstage

oc create secret generic dynamic-plugins-npmrc --from-file ~/.npmrc

helm repo add helm repo add openshift-helm-charts https://charts.openshift.io
helm upgrade -i backstage openshift-helm-charts/redhat-developer-hub --version 1.1.0 -f backstage.yaml

| Backstage | https://backstage-developer-hub-backstage.apps-crc.testing/ |
| -- | -- |

If you want to run the demo fully offline later, run through all `yarn add` steps in individual demos (to cache the NPM packages from NPMJS registry). Additionally you will need the `backstage` chart locally as well. Pull it via:

In [3]:
!helm pull openshift-helm-charts/redhat-developer-hub --version 1.1.0